**DATA IMPORT AND AUGMENTATION**

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate

In [ ]:
#Create new path to store augmented images
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
def load_data(path):
  train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
  train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

  test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
  test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

  return (train_x, train_y), (test_x,test_y)

In [ ]:
def augment_data(images, masks, save_path, augment=True):
  size=(512,512)

  #Now we loop over image and mask
  for idx, (x,y) in tqdm(enumerate(zip(images,masks)), total=len(images)):
    #extracting name of the image
    name = (x.split('/')[-1]).split(".")[0]

    #Reading the image and mask
    #for image we use cv2 and for the mask we use imageio

    x = cv2.imread(x, cv2.IMREAD_COLOR)
    y = imageio.mimread(y)[0]
    if augment == True:
      aug = HorizontalFlip(p=1.0)
      augmented = aug(image=x, mask=y)
      x1 = augmented["image"]
      y1 = augmented["mask"]

      aug = VerticalFlip(p=1.0)
      augmented = aug(image=x, mask=y)
      x2 = augmented["image"]
      y2 = augmented["mask"]

      aug = Rotate(limit=45, p=1.0)
      augmented = aug(image=x, mask=y)
      x3 = augmented["image"]
      y3 = augmented["mask"]

      X = [x, x1, x2, x3]
      Y = [y, y1, y2, y3]

    else:
      X = [x]
      Y = [y]

    index = 0
    for i, m in zip(X,Y):
      i = cv2.resize(i, size)
      m = cv2.resize(m, size)

      tmp_image_name = f"{name}_{index}.png"
      tmp_mask_name = f"{name}_{index}.png"

      image_path = os.path.join(save_path, "image", tmp_image_name)
      mask_path = os.path.join(save_path, "mask", tmp_mask_name)  

      cv2.imwrite(image_path, i)
      cv2.imwrite(mask_path, m)  

      index+=1


In [ ]:
if __name__ == "main":
  np.random.seed(42)

In [ ]:
##Load the data
data_path = "/content/drive/MyDrive/Retina Vessel Segmentation/DRIVE/"
(train_x, train_y), (test_x,test_y) = load_data(data_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 20 - 20
Test: 20 - 20


In [ ]:
#Now we create directories to save our augmented data
create_dir("new_data/train/image/")
create_dir("new_data/train/mask/")
create_dir("new_data/test/image/")
create_dir("new_data/test/mask/")

In [ ]:
#Data augmentation
augment_data(train_x,train_y,"new_data/train/",augment=True)

100%|██████████| 20/20 [00:19<00:00,  1.03it/s]


In [ ]:
augment_data(test_x,test_y,"new_data/test/",augment=False)

100%|██████████| 20/20 [00:18<00:00,  1.09it/s]


**CREATING U-NET ARCHITECTURE**

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class conv_block(nn.Module):
  def __init__(self, in_c, out_c):
    super().__init__()

    self.conv1 = nn.Conv2d(in_c,out_c,kernel_size=3,padding=1)
    self.bn1 = nn.BatchNorm2d(out_c)

    self.conv2 = nn.Conv2d(out_c,out_c,kernel_size=3,padding=1)
    self.bn2 = nn.BatchNorm2d(out_c)

    self.relu = nn.ReLU()

  def forward(self, inputs):
    x = self.conv1(inputs)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)

    return x


class encoder_block(nn.Module):
  def __init__(self, in_c,out_c):
    super().__init__()

    self.conv = conv_block(in_c, out_c)
    self.pool = nn.MaxPool2d((2,2))

  def forward(self, inputs):
    x = self.conv(inputs)
    p = self.pool(x)

    return x,p

class decoder_block(nn.Module):
  def __init__(self, in_c, out_c):
    super().__init__()

    self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
    #Number of input channels will be outputchannles*2
    self.conv = conv_block(out_c+out_c, out_c)

  def forward(self, inputs, skip):
    x = self.up(inputs)
    x = torch.cat([x, skip], axis=1)
    x = self.conv(x)
    return x
   


class build_unet(nn.Module):
    def __init__(self):
        super().__init__()

        ##Encoder 
        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)

        ##Bottleneck
        self.b = conv_block(512, 1024)  

        ##Decoder
        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)


        ##Classifier 
        #To generate the final segmentation
        self.outputs = nn.Conv2d(64,1, kernel_size=1, padding=0)

    
    def forward(self, inputs):
        ##Encoder 
        #The output from the encoder layer acts as a skip connection for the decoder layer
        #s1 stands for skip connections
        #p1 stands for pooling output

        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        #Bottleneck
        b = self.b(p4)
        #print(s1.shape, s2.shape, s3.shape, s4.shape)
        #print(b.shape)

        ##Decoder
        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        #Classifier
        outputs = self.outputs(d4)
        return outputs



if __name__ == "__main__":
  x = torch.randn((2,3,512,512))
  f = build_unet()
  y=f(x)
  print(y.shape)

torch.Size([2, 1, 512, 512])


**LOSS**

In [ ]:
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

**UTILS**

In [ ]:
import os
import time
import random
import numpy as np
import cv2
import torch

##Seeding the randomness
def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

##Create a directory
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

##Calculate the time taken
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

**TRAINING**

In [ ]:
import os
import time
from glob import glob

from torch.utils.data import DataLoader

In [ ]:
if __name__ == "__main__":
  #Seeding
  seeding(42)

  #Saving the checkpoints and other files
  create_dir("files")

  #load the dataset
  train_x = sorted(glob("/content/new_data/train/image/*"))
  train_y = sorted(glob("/content/new_data/train/mask/*"))

  valid_x = sorted(glob("/content/new_data/test/image/*"))
  valid_y = sorted(glob("/content/new_data/test/mask/*"))

  data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid:{len(test_x)}"
  print(data_str)

Dataset Size:
Train: 80 - Valid:20


In [ ]:
##Hyperparameters
H = 512
W = 512
size = (H,W)
batch_size = 2
num_epochs = 300
lr = 1e-4
checkpoint_path = "files/checkpoint.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


**DATASET AND LOADER**

In [ ]:
import os 
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset

In [ ]:
class DriveDataset(Dataset):
  def __init__(self, images_path, masks_path):
    self.images_path = images_path
    self.masks_path = masks_path
    self.n_samples = len(images_path)

  def __getitem__(self, index):
    #Reading the image
    #Read the image in RGB format
    image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
    #Normalize the image by dividing with 255
    image = image/255.0
    #Transpose the image since it had channel last approach and for pytorch we need channel first
    image = np.transpose(image, (2,0,1))
    #Then we convert it into float32
    image = image.astype(np.float32)
    #Then we convert it to a torch tensor
    image = torch.from_numpy(image)

    #Reading the mask
    #We do not need transpose for mask
    mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=0)##(1,512,512)
    mask = mask.astype(np.float32)
    mask = torch.from_numpy(mask)

    return image, mask

  def __len__(self):
    return self.n_samples


In [ ]:
train_dataset = DriveDataset(train_x, train_y)
valid_dataset = DriveDataset(valid_x, valid_y)

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers=2
)

valid_loader = DataLoader(
    dataset = valid_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers=2
)

In [ ]:
##Initialising the model
model = build_unet()
model = model.to(device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min', patience=5, verbose=True)
loss_fn = DiceBCELoss()

In [ ]:
##Specify the training function
def train(model, loader, optimizer, loss_fn, device):
  epoch_loss = 0.0
  model.train()
  for x,y in tqdm(loader):
    x = x.to(device, dtype=torch.float32)
    y = y.to(device, dtype=torch.float32)

    #Optimizer put to zero grad
    optimizer.zero_grad()
    #Making a prediction
    y_pred = model(x)
    #Calculating loss
    loss = loss_fn(y_pred, y)
    #Backward step
    loss.backward()
    #Optimizer step
    optimizer.step()
    #Getting the loss for that epoch
    epoch_loss+=loss.item()

  #Calculating mean loss
  epoch_loss = epoch_loss/len(loader)
  return epoch_loss

In [ ]:
##Specify the evaluate function
def evaluate(model, loader, loss_fn, device):
  epoch_loss = 0.0
  model.eval()
  with torch.no_grad():
    for x,y in tqdm(loader):
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.float32)

      y_pred = model(x)
      loss = loss_fn(y_pred,y)

      epoch_loss+=loss.item()

  #Calculating mean loss
  epoch_loss = epoch_loss/len(loader)
  return epoch_loss

In [ ]:
##Training the model
best_valid_loss = float("inf")
for epoch in range(num_epochs):
  start_time = time.time()
  train_loss = train(model, train_loader, optimizer, loss_fn, device)
  valid_loss = evaluate(model, valid_loader, loss_fn, device)

  #Saving the model
  if valid_loss < best_valid_loss:
    data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:4f}"
    print(data_str)

    best_valid_loss = valid_loss
    torch.save(model.state_dict(), checkpoint_path)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
  data_str += f'\tTrain Loss: {train_loss:.3f}\n'
  data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
  print(data_str)

100%|██████████| 10/10 [00:02<00:00,  3.60it/s]


Valid loss improved from inf to 1.344711
Epoch: 01 | Epoch Time: 0m 39s
	Train Loss: 1.166
	 Val. Loss: 1.345



100%|██████████| 10/10 [00:02<00:00,  3.46it/s]


Valid loss improved from 1.3447 to 1.005763
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 0.981
	 Val. Loss: 1.006



100%|██████████| 10/10 [00:02<00:00,  3.49it/s]


...

Epoch: 298 | Epoch Time: 0m 33s
	Train Loss: 0.142
	 Val. Loss: 0.412



100%|██████████| 10/10 [00:02<00:00,  3.44it/s]


Epoch: 299 | Epoch Time: 0m 33s
	Train Loss: 0.142
	 Val. Loss: 0.410



100%|██████████| 10/10 [00:02<00:00,  3.47it/s]

Epoch: 300 | Epoch Time: 0m 33s
	Train Loss: 0.142
	 Val. Loss: 0.411



**CHECKING ACCURACY**

In [ ]:
import os, time
from operator import add
import numpy as np
from glob import glob
import cv2
from tqdm import tqdm
import imageio
import torch
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


def calculate_metrics(y_true, y_pred):
    ##Ground truth
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    ##Prediction 
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)
    return mask

if __name__ == "__main__":
    ##Seeding 
    seeding(42)

    ##Folders
    create_dir("results")

    ##Load dataset 
    test_x = sorted(glob("/content/new_data/test/image/*"))
    test_y = sorted(glob("/content/new_data/test/mask/*"))

    ##Hyperparameters 
    H = 512
    W = 512
    size = (W, H)
    checkpoint_path = "files/checkpoint.pth"

    ##Load the checkpoint 
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = build_unet()
    model = model.to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()

    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0]
    time_taken = []

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        ##Extract the name 
        name = x.split("/")[-1].split(".")[0]

        ##Reading image 
        image = cv2.imread(x, cv2.IMREAD_COLOR) ## (512, 512, 3)
        ## image = cv2.resize(image, size)
        x = np.transpose(image, (2, 0, 1))      ## (3, 512, 512)
        x = x/255.0
        x = np.expand_dims(x, axis=0)           ## (1, 3, 512, 512)
        x = x.astype(np.float32)
        x = torch.from_numpy(x)
        x = x.to(device)

        ##Reading mask 
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
        ## mask = cv2.resize(mask, size)
        y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
        y = y/255.0
        y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
        y = y.astype(np.float32)
        y = torch.from_numpy(y)
        y = y.to(device)

        with torch.no_grad():
            ##Prediction and Calculating FPS 
            start_time = time.time()
            pred_y = model(x)
            pred_y = torch.sigmoid(pred_y)
            total_time = time.time() - start_time
            time_taken.append(total_time)


            score = calculate_metrics(y, pred_y)
            metrics_score = list(map(add, metrics_score, score))
            pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
            pred_y = np.squeeze(pred_y, axis=0)     ## (512, 512)
            pred_y = pred_y > 0.5
            pred_y = np.array(pred_y, dtype=np.uint8)

        ##Saving masks 
        ori_mask = mask_parse(mask)
        pred_y = mask_parse(pred_y)
        line = np.ones((size[1], 10, 3)) * 128

        cat_images = np.concatenate(
            [image, line, ori_mask, line, pred_y * 255], axis=1
        )
        cv2.imwrite(f"results/{name}.png", cat_images)

    jaccard = metrics_score[0]/len(test_x)
    f1 = metrics_score[1]/len(test_x)
    recall = metrics_score[2]/len(test_x)
    precision = metrics_score[3]/len(test_x)
    acc = metrics_score[4]/len(test_x)
    print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

    fps = 1/np.mean(time_taken)
    print("FPS: ", fps)

100%|██████████| 20/20 [00:07<00:00,  2.58it/s]

Jaccard: 0.6625 - F1: 0.7967 - Recall: 0.7848 - Precision: 0.8138 - Acc: 0.9652
FPS:  137.88162894975608
